# Exploring and Coercing Data

### Introduction

Over the next couple of lessons, we'll talk through coercing our data in pandas.  Our data -- and quality data -- is difficult to come by.  So oftentimes, we may need to clean it before performing analysis on it.

In general, with cleaning our data our main task is to find data that are strings, and convert it to something more useful -- like something numeric or a datetime.  We'll also see a function for pulling out multiple attributes from a datetime. 

### DataTypes in Pandas

Let's take a look at the table below regarding datatypes in Pandas.

|  Pandas dtype |  Python Type | Use |
|---|---|---|
|object|string|text|
|int64/float64| int/float   | numbers|
|datetime64| NA   |Dates and Times|
|bool|bool   |True/False|
|category| NA |Finite list of text values|

For data analysis, we'll mainly be focusing on the first three datatypes.  The datatype `object` corresponds to a string in Python, and a lot of the work in coercing data involves changing a column from a type of object to a different an int/float or datetime.

For this lesson, let's work with our SAT score data up some data about [NYC SAT scores](https://data.cityofnewyork.us/Education/2012-SAT-Results/f9bf-2cp4), drawn from the [NYC Open data](https://opendata.cityofnewyork.us/) website.

In [4]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/jigsawlabs-student/introductory-pandas/master/nyc_hs_sat.csv', index_col = 0)


# to make things more interesting, we also alter some of the data
columns = ['reading_avg', 'math_avg', 'writing_score']
df[columns] = df[columns].astype('object')
str_cols = df[columns].apply(lambda x: x.map(str))
df = df.drop(columns = columns)
sat_df = pd.concat([df, str_cols], axis = 1)

> Press shift + enter on the cell above and we can get started to load the data and we can get started.

Typically, cleaning the data happens at multiple points in the data analysis process.  But it makes sense to do some data exploration first to decide what data is worth cleaning.

### Exploring Our Data

This time when exploring our dataset, we are doing a different kind of data exploration.  Generally, we are seeing the type of data we are working with, so that we can get to cleaning our data of type object.

To do this, we'll use two main methods `db.dtypes` and `df.select_dtypes`.

* `dtypes`

We can call the dtypes method directly on our pandas dataframe.

In [5]:
sat_df.dtypes

dbn                     object
name                    object
num_test_takers        float64
boro                    object
total_students           int64
graduation_rate        float64
attendance_rate        float64
college_career_rate    float64
reading_avg             object
math_avg                object
writing_score           object
dtype: object

The `dtypes` method lists the column name and corresponding datatype for each column.  We can see that a lot of these columns are of type object that we may be able to change to be numeric.  

* `df.select_dtypes`

To narrow in on those series we should coerce, we can use the `select_dtypes` method.

In [6]:
sat_objects_df = sat_df.select_dtypes('object')
sat_objects_df[:2]

,dbn,name,boro,reading_avg,math_avg,writing_score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,M,355.0,404.0,363.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,M,383.0,423.0,366.0


So we see that there are a number of columns that are currently not numeric, but could be good to include in our model.  For example, `reading_avg`, `math_avg`, `writing_score`, and `boro`.

* exclude

If we want to also see the columns that are currently **not** of type object, and thus may be ready for our model, we can find that by using `select_dtypes` to identify the columns that are not of type object.

In [6]:
except_objects_df = sat_df.select_dtypes(exclude = ['object'])
except_objects_df[:2]

,num_test_takers,total_students,graduation_rate,attendance_rate,college_career_rate
0,29.0,171,0.66,0.87,0.36
1,91.0,465,0.90,0.93,0.70


So these columns are not of type object, and look like they are good to go as features of our model.

### Changing the DataType of Columns

Now that we have identified the columns that we may wish to change -- with the `dtypes` and `select_dtypes` methods, let's move onto coercing some of these columns.

Let's start by taking another look at the columns that are currently of type object.

In [55]:
sat_df.select_dtypes('object')[:2]

,dbn,name,boro,reading_avg,math_avg,writing_score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,M,355.0,404.0,363.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,M,383.0,423.0,366.0


The column `reading_avg` looks like it could be predictive of our `math_avg` target, so let's try to make the column numeric.  Currently, the column is of type `object`, and if we look, we see that each of the entries are a string. 

In [8]:
sat_df.reading_avg.dtype

dtype('O')

In [9]:
sat_df.reading_avg[0]

'355.0'

Now if we change the data to be of type numeric, we can eventually use this data as a feature in our model.

In [19]:
reading = sat_df.reading_avg.astype('float64')
reading.dtype

dtype('float64')

In [11]:
reading[0]

355.0

Now that we have a series of data in an integer format, we can replace the original `sat_df` column to be our new `reading`, coerced, column.

In [16]:
sat_df_reading_int = sat_df.assign(reading_avg = reading)
sat_df_reading_int[:1]

,dbn,name,num_test_takers,boro,total_students,graduation_rate,attendance_rate,college_career_rate,reading_avg,math_avg,writing_score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29.0,M,171,0.66,0.87,0.36,355.0,404.0,363.0


> So we just used the `astype` method to specify the datatype that the column should be.  Then, we replaced the old column with with the new coerced column with the `assign` method.

Ok, Let's see our progress, by checking that there is one fewer column of type `object`.

In [17]:
sat_df_reading_int.select_dtypes('object')[:2]

,dbn,name,boro,math_avg,writing_score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,M,404.0,363.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,M,423.0,366.0


So in general, we can move through this process:
    
1. Identify the columns to clean with df.select_dtypes('object')
2. Convert the data to be of type float (if possible) 
    * `reading = sat_df.reading_avg.astype('float64')`
3. Then use `assign` to update the dataframe with the coerced column
    * `sat_df_reading_int = sat_df.assign(reading_avg = reading)`

### Coercing DateTime Data

Now there's more work to do with our SAT dataset, but we'll leave that for you in the next lab.  For now, let's move onto working with another type of data, datetimes.  To do so, we'll use some revenue data from Max's Wine Bar in Texas.  We currently have the data stored in JSON.  Let's load it up.

In [28]:
max_revenue_df = pd.read_json('https://raw.githubusercontent.com/jigsawlabs-student/introductory-pandas/master/max-revenue.json')

In [29]:
max_revenue_df[:2]

,total_receipts,end_date
0,56182,2016-12-31T00:00:00.000
1,9400,2017-08-31T00:00:00.000


In the dataframe above, `total_receipts` the represents the revenue earned from alcohol in a month, and the `end_date` is the month in which that revenue was earned.  So the first row indicates that `56182` was earned in the month of December 2016.  

Let's say we want to get a sense of average revenue earned per month and average revenue earned year.  Our first step would be to make sure our end_date is of type `datetime`.  Currently it isn't.

In [30]:
max_revenue_df.dtypes

total_receipts     int64
end_date          object
dtype: object

So let's change it.

In [31]:
end_date = max_revenue_df.end_date.astype('datetime64')
end_date[:2]

0   2016-12-31
1   2017-08-31
Name: end_date, dtype: datetime64[ns]

Another way that we can do this is using the `pd.to_datetime` method.

In [32]:
end_date = pd.to_datetime(max_revenue_df.end_date)
end_date[:2]

0   2016-12-31
1   2017-08-31
Name: end_date, dtype: datetime64[ns]

Ok, now that our data is of type `datetime`, we can call methods to extract the month, weekday, or year from each entry.

In [25]:
end_date[0].month

12

But oftentimes, we'll just want to quickly extract all of the possible information from our datetime.  For that we can use the `add_datepart` function, which comes from the [Fastai](https://docs.fast.ai/tabular.core.html) library.

In [35]:
import numpy as np
import re
def add_datepart(df, fldnames, drop=False, time=False, errors="raise"):
    """add_datepart converts a column of df from a datetime64 to many columns containing
    the information from the date. This applies changes inplace."""
    if isinstance(fldnames,str):
        fldnames = [fldnames]
    for fldname in fldnames:
        fld = df[fldname]
        fld_dtype = fld.dtype
        if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
            fld_dtype = np.datetime64

        if not np.issubdtype(fld_dtype, np.datetime64):
            df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True, errors=errors)
        targ_pre = re.sub('[Dd]ate$', '', fldname)
        attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear']
        if time: attr = attr + ['Hour', 'Minute', 'Second']
        for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
        df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
        if drop: df.drop(fldname, axis=1, inplace=True)

Once we have the function (which we'll often give to you, or you can just google), we can use it like so.

In [36]:
max_revenue_dated = max_revenue_df.copy()
fldnames = ['end_date']
add_datepart(max_revenue_dated, fldnames)

/var/folders/56/76xkbl_1039gpjh3w47v01980000gn/T/ipykernel_35384/173388946.py:19: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())


In [37]:
max_revenue_dated[:2]

,total_receipts,end_date,end_Year,end_Month,end_Week,end_Day,end_Dayofweek,end_Dayofyear,end_Elapsed
0,56182,2016-12-31,2016,12,52,31,5,366,1483142400
1,9400,2017-08-31,2017,8,35,31,3,243,1504137600


so we can see that we just passed through the dataframe, and the `fldnames` and the function added various columns for us -- extracting the related year, month, etc from the datetime.  The `Elapsed` column at the end shows total time elapsed from a date in the past, so can be good for sorting the data chronologically.

### Summary

In this lesson, we saw how to coerce our data into formats that are not objects. 

A. Convert our objects

 We did this using the following process.


   1. Identify the columns to clean with `select_dtypes` 

In [43]:
df.select_dtypes('object')[:1]

,dbn,name,boro
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,M


Convert the data to be of type float (if possible) with `astype`

In [41]:
reading = sat_df.reading_avg.astype('float64')

3. Then use `assign` to update the dataframe with the coerced column

In [42]:
sat_df_reading_int = sat_df.assign(reading_avg = reading)

B. Extract info from our datetimes

For this we use the `add_datepart` function, identifying the fields that we would like to extract information from.

In [ ]:
max_revenue_dated = max_revenue_df.copy()
fldnames = ['end_date']
add_datepart(max_revenue_dated, fldnames)